### JSON 데이터 전처리 후 JSON파일을 데이터프레임으로 변경
- 임의로 0으로 설정하긴 했는데 이건 상의 후 결정
- 0으로 하면 등급이 0 등급도 있고 측정값이 0인 것도 있어서 영향을 줄 것 같다.
- 어떻게 해야할까?

In [ ]:
# JSON파일을 DataFrame으로 변경
# 데이터 프레임에 값이 딕셔너리를 문자열로 만들어 값에 들어가 있어 문자열 -> 딕셔너리
import os
import json
import pandas as pd
from ultralytics import YOLO
def create_labeling_dataframe(base_path):
    data = []
    for split in ['Validation', 'Training']:
        label_path = os.path.join(base_path, split, '02.라벨링데이터')
        if not os.path.exists(label_path):
            print(f"경로가 존재하지 않습니다: {label_path}")
            continue
        for file in os.listdir(label_path):
            if file.endswith('.json'):
                with open(os.path.join(label_path, file), 'r') as f:
                    json_data = json.load(f)
                    row = {
                        'split': split,
                        'info': json.dumps(json_data.get('info', {})),
                        'images': json.dumps(json_data.get('images', {})),
                        'annotations': json.dumps(json_data.get('annotations', {})),
                        'equipment': json.dumps(json_data.get('equipment', {}))
                    }
                    data.append(row)
    
    df = pd.DataFrame(data)
    
    # JSON 문자열을 딕셔너리로 변환
    for col in ['info', 'images', 'annotations', 'equipment']:
        df[col] = df[col].apply(lambda x: x if isinstance(x, dict) else json.loads(x) if isinstance(x, str) else x)
    
    return df
  
base_path = 'data'
df = create_labeling_dataframe(base_path)

df.head(3)

In [ ]:
# 5. 데이터 프레임 전처리

# 데이터 프레임 None 값을 {}로 변경
# 딕셔너리 안에 None 값을 0으로 바꾸는 함수 정의
# acne -> 0이 없는 거
def replace_none_with_zero(d):
    if isinstance(d, dict):
        return {k: (0 if v is None else v) for k, v in d.items()}
    return d

# 데이터 프레임의 모든 셀에 대해 함수 적용
df = df.applymap(replace_none_with_zero)

# 'equipment' 컬럼에 null 값을 빈 딕셔너리로 변경
df['equipment'] = df['equipment'].apply(lambda x: {} if pd.isna(x) else x)
df.head(3)

In [ ]:
len(df)

In [ ]:
df.iloc[0]['info']

In [ ]:
df.to_csv('data.csv', index=False)

### YOLOv8을 사용한 얼굴 부위 분류 모델 학습

In [1]:
import os
import json
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from ultralytics import YOLO
from PIL import Image
import shutil
import ast
import matplotlib.pyplot as plt

In [2]:
# YAML 파일 생성 함수
def create_yolo_yaml(base_path):
    yaml_content = f"""
path: {base_path}
train: Training/images
val: Validation/images
nc: 9
names: ['forehead', 'glabella', 'left_eye', 'right_eye', 'left_cheek', 'right_cheek', 'nose', 'mouth', 'chin']

train_labels: Training/labels
val_labels: Validation/labels
"""
    yaml_path = os.path.join(base_path, 'dataset.yaml')
    with open(yaml_path, 'w', encoding='utf-8') as f:
        f.write(yaml_content)
    return yaml_path

In [3]:
# 손상된 이미지와 라벨링 같이 제거
def remove_corrupted_images(folder_path):
    corrupted_images = []
    for filename in os.listdir(folder_path):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            file_path = os.path.join(folder_path, filename)
            try:
                with Image.open(file_path) as img:
                    img.verify()
            except Exception as e:
                print(f"손상된 이미지 파일 발견: {filename}")
                corrupted_images.append(filename)
                os.remove(file_path)
    return corrupted_images

In [4]:
# yolo형식으로 데이터 전처리 함수
def prepare_yolo_data(df, base_path):
    train_image_path = os.path.join(base_path, 'Training', 'images')
    val_image_path = os.path.join(base_path, 'Validation', 'images')
    train_label_path = os.path.join(base_path, 'Training', 'labels')
    val_label_path = os.path.join(base_path, 'Validation', 'labels')

    # 모든 필요한 폴더가 이미 존재하는지 확인
    if all(os.path.exists(path) for path in [train_image_path, val_image_path, train_label_path, val_label_path]):
        print("기존 데이터 폴더를 사용합니다.")
        return train_image_path, val_image_path, train_label_path, val_label_path

    # 폴더가 없는 경우 새로 생성
    os.makedirs(train_image_path, exist_ok=True)
    os.makedirs(val_image_path, exist_ok=True)
    os.makedirs(train_label_path, exist_ok=True)
    os.makedirs(val_label_path, exist_ok=True)

    print("Training 데이터에서 손상된 이미지 제거 중...")
    remove_corrupted_images(train_image_path)
    print("Validation 데이터에서 손상된 이미지 제거 중...")
    remove_corrupted_images(val_image_path)

    processed_count = 0
    for index, row in df.iterrows():
        try:
            info = row['info']
            images = row['images']

            filename = info['filename']
            facepart = images['facepart']
            bbox = images['bbox']
            width = images['width']
            height = images['height']

            src_img_path = os.path.join(base_path, row['split'], '01.원천데이터', filename)
            dst_img_path = os.path.join(train_image_path if 'Training' in row['split'] else val_image_path, filename)

            if not os.path.exists(src_img_path):
                print(f"{index}번 행을 건너뜁니다. 이미지 파일이 존재하지 않습니다: {filename}")
                continue

            # 이미지 파일 복사
            shutil.copy2(src_img_path, dst_img_path)

            if not all([filename, facepart is not None, width, height, bbox]):
                print(f"{index}번 행을 건너뜁니다. 파일에 대한 필수 정보가 누락되었습니다: {filename}")
                continue

            if (bbox == ['None', 'None', 'None', 'None']) or not all(isinstance(b, (int, float)) and b is not None for b in bbox):
                print(f"{index}번 행을 건너뜁니다. 유효하지 않은 bbox 값입니다. 파일: {filename}")
                continue

            bbox = [float(b) for b in bbox]
            width = float(width)
            height = float(height)

            x_center = (bbox[0] + bbox[2]) / 2 / width
            y_center = (bbox[1] + bbox[3]) / 2 / height
            bbox_width = (bbox[2] - bbox[0]) / width
            bbox_height = (bbox[3] - bbox[1]) / height

            yolo_format = f"{facepart} {x_center} {y_center} {bbox_width} {bbox_height}\n"

            label_path = train_label_path if 'Training' in row['split'] else val_label_path
            label_filename = os.path.splitext(filename)[0] + '.txt'

            with open(os.path.join(label_path, label_filename), 'a') as f:
                f.write(yolo_format)

            processed_count += 1

        except Exception as e:
            print(f"{index}번 행 처리 중 오류 발생: {str(e)}")
            print(f"행 데이터: {row}")

    print(f"총 {processed_count}개의 이미지를 처리했습니다.")
    return train_image_path, val_image_path, train_label_path, val_label_path

In [5]:
# 성능 시각화 함수
import matplotlib.pyplot as plt

def plot_performance(results):
    metrics = ['box_loss', 'cls_loss', 'dfl_loss', 'mAP50', 'mAP50-95', 'precision', 'recall']

    fig, axs = plt.subplots(3, 3, figsize=(20, 20))
    axs = axs.ravel()  # 2D array를 1D array로 변환

    for i, metric in enumerate(metrics):
        ax = axs[i]
        if metric in results.results_dict:
            ax.plot(results.results_dict[metric], label=f'train_{metric}')
            if f'val_{metric}' in results.results_dict:
                ax.plot(results.results_dict[f'val_{metric}'], label=f'val_{metric}')
            ax.set_title(f'{metric.replace("_", " ").capitalize()}')
            ax.set_xlabel('Epoch')
            ax.set_ylabel('Value')
            ax.legend()

    # 사용하지 않는 서브플롯 제거
    for j in range(i+1, 9):
        fig.delaxes(axs[j])

    plt.tight_layout()
    plt.savefig('training_performance.png')
    plt.close()

    print("성능 그래프가 'training_performance.png'로 저장되었습니다.")

In [6]:
# YOLO 모델 학습
def train_yolo_model(df, base_path):
    train_image_path, val_image_path, train_label_path, val_label_path = prepare_yolo_data(df, base_path)

    yaml_path = create_yolo_yaml(base_path)

    model = YOLO('yolov8n.yaml')
    results = model.train(data=yaml_path, epochs=30, imgsz=640)

    model.save('yolov8_facepart_model.pt')

    plot_performance(results)

    return model, results

In [ ]:
# 함수들 실행
if __name__ == "__main__":
    base_path = os.path.abspath('data')
    
    df = pd.read_csv(base_path + '/json to df.csv')
    
    # csv로 불러오니 딕셔너리가 문자열로 다 바뀌어서 나와 딕셔너리로 변환
    for col in ['info', 'images', 'annotations', 'equipment']:
        df[col] = df[col].apply(lambda x: ast.literal_eval(x) if x else None)
        
    trained_model, training_results = train_yolo_model(df, base_path)
    
    if trained_model:
        print("모델 학습이 완료되었습니다.")
        print("학습 결과:", training_results)
        print("학습된 모델이 'yolov8_facepart_model.pt'로 저장되었습니다.")
    else:
        print("모델 학습에 실패했습니다.")

Training 데이터에서 손상된 이미지 제거 중...
Validation 데이터에서 손상된 이미지 제거 중...
58번 행을 건너뜁니다. 파일에 대한 필수 정보가 누락되었습니다: 0001_01_R30.jpg
75번 행을 건너뜁니다. 파일에 대한 필수 정보가 누락되었습니다: 0001_02_L.jpg
85번 행을 건너뜁니다. 파일에 대한 필수 정보가 누락되었습니다: 0001_02_R.jpg
102번 행을 건너뜁니다. 파일에 대한 필수 정보가 누락되었습니다: 0001_03_L.jpg
112번 행을 건너뜁니다. 파일에 대한 필수 정보가 누락되었습니다: 0001_03_R.jpg
175번 행을 건너뜁니다. 파일에 대한 필수 정보가 누락되었습니다: 0004_01_R30.jpg
192번 행을 건너뜁니다. 파일에 대한 필수 정보가 누락되었습니다: 0004_02_L.jpg
202번 행을 건너뜁니다. 파일에 대한 필수 정보가 누락되었습니다: 0004_02_R.jpg
210번 행을 건너뜁니다. 파일에 대한 필수 정보가 누락되었습니다: 0004_03_F.jpg
211번 행을 건너뜁니다. 파일에 대한 필수 정보가 누락되었습니다: 0004_03_F.jpg
214번 행을 건너뜁니다. 파일에 대한 필수 정보가 누락되었습니다: 0004_03_F.jpg
217번 행을 건너뜁니다. 파일에 대한 필수 정보가 누락되었습니다: 0004_03_L.jpg
219번 행을 건너뜁니다. 파일에 대한 필수 정보가 누락되었습니다: 0004_03_L.jpg
221번 행을 건너뜁니다. 파일에 대한 필수 정보가 누락되었습니다: 0004_03_L.jpg
223번 행을 건너뜁니다. 파일에 대한 필수 정보가 누락되었습니다: 0004_03_L.jpg
226번 행을 건너뜁니다. 파일에 대한 필수 정보가 누락되었습니다: 0004_03_R.jpg
229번 행을 건너뜁니다. 파일에 대한 필수 정보가 누락되었습니다: 0004_03_R.jpg
231번 행을 건너뜁니다. 파일에 대한 필수 정보가 누락되었습니다: 0004_03_R.jpg

train: Scanning C:\workspace_multi07\web\15_final project\data\Training\labels... 11154 images, 0 backgrounds, 1 corrup

train: WARNING  C:\workspace_multi07\web\15_final project\data\Training\images\0972_01_L15.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0435]


train: New cache created: C:\workspace_multi07\web\15_final project\data\Training\labels.cache


val: Scanning C:\workspace_multi07\web\15_final project\data\Validation\labels... 1387 images, 0 backgrounds, 1 corrupt


val: WARNING  C:\workspace_multi07\web\15_final project\data\Validation\images\0615_01_L30.jpg: ignoring corrupt image/label: image file is truncated (96 bytes not processed)
val: New cache created: C:\workspace_multi07\web\15_final project\data\Validation\labels.cache
Plotting labels to runs\detect\train2\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000769, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added 
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to runs\detect\train2
Starting training for 30 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       1/30         0G      3.851      4.514      4.229        262        640:   1%|▏         | 10/698 [05:21<5:39:46, 

### 딥러닝 및 나머지 코드

In [ ]:
import os
import json
import numpy as np
import pandas as pd
from PIL import Image
from ultralytics import YOLO
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, GlobalAveragePooling2D
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import logging
import yaml

In [ ]:
# 로깅 설정
logging.basicConfig(filename='skin_analysis.log', level=logging.INFO, 
                    format='%(asctime)s:%(levelname)s:%(message)s')

In [ ]:
# YOLO 모델 로드 함수
def load_yolo_model(model_path):
    return YOLO(model_path)

In [ ]:
# 이미지를 224x224 크기로 로드 (ResNet50의 기본 입력 크기), 픽셀 값을 0~1 사이로 정규화
#  prepare_skin_data 함수에서 각 facepart 이미지를 로드할 때 사용
# 이미지 로드 및 전처리 함수
def load_and_preprocess_image(img_path):
    img = load_img(img_path, target_size=(224, 224))  # ResNet50의 기본 입력 크기
    img_array = img_to_array(img)
    img_array = img_array / 255.0  # 정규화
    return img_array

In [ ]:
df.iloc[0]['images']['facepart']

In [ ]:
# 피부 진단 데이터 준비 함수
def prepare_skin_data(df, facepart, base_path):
    # DataFrame에서 'Training' 데이터와 특정 facepart에 해당하는 데이터만 선택
    skin_data = df[(df['split'] == 'Training') & (df['images'].apply(lambda x: json.loads(x)['facepart'] == facepart))]
    
    X = []  # 이미지 데이터를 저장할 리스트
    y = []  # 라벨 데이터를 저장할 리스트
    
    for _, row in skin_data.iterrows():
        info = json.loads(row['info'])
        annotations = json.loads(row['annotations'])
        equipment = json.loads(row['equipment'])

        if facepart == 0:
            # facepart = 0은 얼굴 전체 이미지여서 원본 데이터 사용
            img_path = os.path.join(base_path, 'Training/01.원천데이터', info['filename'])
            # 이미지 로드 및 전처리
            img = load_and_preprocess_image(img_path)
            X.append(img)
            
        else:
            # 미리 자른 이미지 파일 경로
            img_path = os.path.join(base_path, 'Training_cropped', str(facepart), info['filename'])
            # 이미지 로드 및 전처리
            img = load_and_preprocess_image(img_path)
            X.append(img)
        
        # 라벨 데이터 준비
        if annotations is not None and equipment is not None:
            features = list(annotations.values()) + list(equipment.values())
            y.append(features)
        else:
            y.append([info['skin_type'], info['sensitive']])
    
    return np.array(X), np.array(y)

In [ ]:
# CNN 모델 생성 함수
def create_cnn_model(input_shape, output_shape):
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        Flatten(),
        Dense(64, activation='relu'),
        Dense(output_shape, activation='softmax')
    ])
    
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
# ResNet50 모델 생성 함수(이건 분류모델   예측(회귀) 모델을 짜야한다)
def create_resnet50_model(input_shape, output_shape):
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    x = Dense(output_shape, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=x)
    
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
# 피부 진단 모델 학습 함수
# 'Training'폴더 데이터만 사용 'Validation'폴더에 있는 데이터들은 서비스 구현 테스트할 때 최종테스트용
def train_skin_models(df, base_path, model_type='cnn'):
    skin_models = {}
    
    for facepart in range(9):
        X, y = prepare_skin_data(df, facepart, base_path)
        
        if len(X) == 0:
            continue
        
        # Training 데이터를 train과 validation으로 분할
        X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state= 0)
        
        input_shape = X_train.shape[1:]
        output_shape = y_train.shape[1]
        
        if model_type == 'cnn':
            model = create_cnn_model(input_shape, output_shape)
        elif model_type == 'resnet50':
            model = create_resnet50_model(input_shape, output_shape)
        else:
            raise ValueError("Invalid model type. Choose 'cnn' or 'resnet50'.")
        
        # 이미지 데이터 증강
        datagen = ImageDataGenerator(
            rotation_range=20,
            width_shift_range=0.2,
            height_shift_range=0.2,
            horizontal_flip=True
        )
        
        # 모델 학습
        model.fit(datagen.flow(X_train, y_train, batch_size=32),
                  epochs=50,
                  validation_data=(X_val, y_val))
        
        skin_models[facepart] = model
    
    return skin_models

In [ ]:
# 이미지에서 얼굴 부위 추출 함수
def extract_faceparts(image, yolo_model):
    results = yolo_model(image)
    faceparts = []
    for r in results:
        boxes = r.boxes
        for box in boxes:
            x1, y1, x2, y2 = box.xyxy[0]
            facepart = image[int(y1):int(y2), int(x1):int(x2)]
            faceparts.append(facepart)
    return faceparts

In [ ]:
# 피부 상태 예측 함수
def predict_skin_condition(image_path, yolo_model, skin_models):
    image = load_img(image_path)
    image = img_to_array(image)
    
    faceparts = extract_faceparts(image, yolo_model)
    
    predictions = {}
    for i, facepart in enumerate(faceparts):
        facepart = load_and_preprocess_image(facepart)
        facepart = np.expand_dims(facepart, axis=0)
        
        if i in skin_models:
            prediction = skin_models[i].predict(facepart)
            predictions[i] = prediction[0]
    
    return predictions

In [ ]:
# 메인 실행 코드
if __name__ == "__main__":
    base_path = os.path.abspath('data') # 데이터 경로

    # 피부 진단 모델 학습 (CNN 사용)
    skin_models_cnn = train_skin_models(df, base_path, model_type='cnn')

    # 피부 진단 모델 학습 (ResNet50 사용)
    skin_models_resnet = train_skin_models(df, base_path, model_type='resnet50')

    # 모델 저장
    for facepart, model in skin_models_cnn.items():
        model.save(f'skin_model_cnn_{facepart}.h5')
    for facepart, model in skin_models_resnet.items():
        model.save(f'skin_model_resnet_{facepart}.h5')

    print("모든 모델 학습 완료")

In [ ]:
    yolo_model_path = 'path/to/your/trained_yolo_model.pt' # 학습한 yolo 모델 path
    # YOLO 모델 로드 (이미 학습된 모델 사용)
    yolo_model = load_yolo_model(yolo_model_path)

    # Validation 데이터셋의 이미지들에 대해 예측 실행
    validation_image_path = os.path.join(base_path, 'Validation', '01.원천데이터')
    for image_file in os.listdir(validation_image_path):
        if image_file.endswith(('.jpg', '.jpeg', '.png')):
            image_path = os.path.join(validation_image_path, image_file)
            predictions = predict_skin_condition(image_path, yolo_model, skin_models_cnn)  # 또는 skin_models_resnet
            print(f"이미지 {image_file}의 예측 결과:", predictions)